# Unigram Modeling

In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
from lxml import html
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer

reviews= []
ratings1= []

for i in range(0,500,20):
    url = "https://www.yelp.com/biz/the-majestic-yosemite-dining-room-yosemite-national-park?start=" + str(i) 
    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(5)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")


    review_blocks = soup.findAll('div',attrs={'class':'review-content'})

    for block in review_blocks:

        review = BeautifulSoup(str(block.findAll('p')),'lxml').get_text()

        # remove extraneous text, line breaks and leading and trailing spaces
        review = review.replace("Was this review …?","")
        review = review.replace("\n","")
        review = review.strip()
        #reviews.append(review)
        words = word_tokenize(review)
        clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
        english_stops = set(stopwords.words('english'))
        clean_words = [word for word in clean_words if word not in english_stops]
        wordnet_lemmatizer = WordNetLemmatizer()
        lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
        reviews.append(lemma_list)


    for rating in review_blocks:
        rating2 = rating.find('div',{'class':"i-stars"})
        if 'i-stars i-stars--regular-1 rating-large' in str(rating2):
            print('neg')
            ratings1.append('neg')
        elif 'i-stars i-stars--regular-2 rating-large' in str(rating2):
            print('neg')
            ratings1.append('neg')  
        elif 'i-stars i-stars--regular-3 rating-large' in str(rating2):
            print('neg')
            ratings1.append('neg') 
        elif 'i-stars i-stars--regular-4 rating-large' in str(rating2):
            print('pos')
            ratings1.append('pos')      
        elif 'i-stars i-stars--regular-5 rating-large' in str(rating2):
            print('pos')
            ratings1.append('pos')     
        else:
            print("idk")

    driver.close()
        
 
    
    

pos
neg
neg
pos
pos
neg
neg
neg
pos
pos
neg
pos
neg
pos
neg
neg
neg
pos
pos
pos
neg
pos
neg
neg
pos
pos
neg
pos
pos
neg
neg
pos
pos
neg
neg
neg
neg
neg
pos
neg
neg
neg
neg
neg
neg
pos
pos
neg
neg
neg
neg
pos
pos
pos
pos
neg
neg
pos
neg
neg
pos
pos
neg
neg
neg
neg
neg
neg
neg
pos
pos
neg
pos
pos
neg
neg
neg
neg
neg
neg
neg
pos
neg
neg
neg
pos
pos
neg
neg
neg
pos
pos
neg
neg
pos
neg
pos
pos
pos
neg
pos
pos
neg
neg
neg
pos
neg
neg
neg
neg
neg
pos
neg
pos
neg
pos
pos
neg
pos
neg
pos
neg
neg
pos
pos
neg
pos
pos
neg
neg
neg
pos
pos
neg
neg
neg
neg
neg
pos
pos
pos
pos
pos
pos
neg
pos
pos
neg
neg
pos
neg
neg
neg
neg
neg
pos
neg
neg
pos
neg
pos
pos
neg
neg
pos
pos
neg
neg
pos
neg
pos
pos
pos
pos
neg
neg
neg
neg
neg
neg
neg
pos
pos
neg
neg
neg
pos
pos
neg
pos
pos
pos
neg
neg
pos
neg
neg
pos
pos
neg
pos
pos
pos
neg
pos
neg
neg
neg
neg
neg
neg
neg
neg
pos
neg
neg
neg
pos
neg
neg
neg
pos
pos
neg
neg
neg
pos
pos
neg
pos
neg
neg
neg
pos
neg
neg
pos
neg
neg
neg
neg
pos
pos
neg
neg
neg
neg
pos
neg
pos


In [13]:
print(len(reviews))

500


In [14]:
print(len(ratings1))

500


In [15]:
combined = zip(reviews,ratings1)

In [16]:
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [17]:
train_data = []

for r, v in combined:
    bag_of_words(r)
    train_data.append((bag_of_words(r),v))
    
    
print(train_data)  

[({'went': True, 'celebrate': True, 'anniversary': True, 'despite': True, 'little': True, 'turned': True, 'off/hassled': True, 'dress': True, 'code': True, 'forgotten': True, 'collared': True, 'shirt': True, 'black': True, 'jacket': True, 'barely': True, 'cut': True, 'seated': True, 'service': True, 'excellent': True, 'reservation': True, '5:30': True, 'first': True, 'sat': True, 'lamb': True, 'ok': True, 'partner': True, 'salmon': True, 'awesome': True, 'kale': True, 'salad': True, 'drink': True, 'good': True, 'slightly': True, 'sweet': True, 'marmalade': True, 'margarita': True, 'gimlet': True, 'dessert': True, 'outstanding': True, 'sorbet': True, 'point': True, 'also': True, 'nice': True, 'surprise': True, 'help': True, 'u': True, 'special': True, 'occasion': True, 'see': True, 'photo': True, 'overall': True, 'great': True, 'experience': True}, 'pos'), ({'expect': True, 'pay': True, 'lot': True, 'mediocre': True, 'food': True, 'awesome': True, 'view': True, 'beautiful': True, 'venue

In [95]:
#shuffling the data
import random
random.shuffle(train_data)
print(len(train_data))

500


In [28]:
train_set, test_set = train_data[350:], train_data[:150]

In [35]:
import nltk
import collections
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [36]:
from nltk.classify import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier.train(train_set, 
                                             binary=True, 
                                             entropy_cutoff=0.8, 
                                             depth_cutoff=5, 
                                             support_cutoff=30)

In [37]:
from nltk.classify import MaxentClassifier

logit_classifier = MaxentClassifier.train(train_set, algorithm='gis', trace=0, max_iter=10, min_lldelta=0.5)

In [39]:
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC

SVM_classifier = SklearnClassifier(SVC(), sparse=False).train(train_set)


In [50]:
from nltk.metrics.scores import (accuracy, precision, recall, f_measure)

#classifier = NaiveBayesClassifier.train(trainfeats)
nbrefset = collections.defaultdict(set)
nbtestset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    nbrefset[label].add(i)
    observed = nb_classifier.classify(feats)
    nbtestset[observed].add(i)
print("NB Recall")
print('neg recall:', recall(nbtestset['neg'], nbrefset['neg']))

refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = logit_classifier.classify(feats)
    testset[observed].add(i)
print("Logit Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = SVM_classifier.classify(feats)
    testset[observed].add(i)
print("SVM Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = dt_classifier.classify(feats)
    testset[observed].add(i)
print("DT Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))


NB Recall
neg recall: 0.648
Logit Recall
neg recall: 0.7117117117117117
SVM Recall
neg recall: None
DT Recall
neg recall: 0.65


# Bigram Modeling

In [230]:
from nltk import bigrams, trigrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

In [231]:
def bag_of_bigrams_words(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)  
    bigrams = bigram_finder.nbest(score_fn, n)  
    return bag_of_words(bigrams)

In [232]:
combined = zip(reviews,ratings1)

In [202]:
train_data2 =[]

for z, e in combined:
    bag_of_bigrams_words(z)
    train_data2.append((bag_of_bigrams_words(z),e))
    
print(train_data2)

[({('5:30', 'first'): True, ('also', 'nice'): True, ('anniversary', 'despite'): True, ('awesome', 'kale'): True, ('barely', 'cut'): True, ('black', 'jacket'): True, ('code', 'forgotten'): True, ('collared', 'shirt'): True, ('cut', 'seated'): True, ('despite', 'little'): True, ('dessert', 'outstanding'): True, ('dress', 'code'): True, ('drink', 'good'): True, ('first', 'sat'): True, ('forgotten', 'collared'): True, ('gimlet', 'dessert'): True, ('good', 'slightly'): True, ('great', 'experience'): True, ('help', 'u'): True, ('jacket', 'barely'): True, ('kale', 'salad'): True, ('lamb', 'ok'): True, ('little', 'turned'): True, ('margarita', 'gimlet'): True, ('marmalade', 'margarita'): True, ('nice', 'surprise'): True, ('occasion', 'see'): True, ('off/hassled', 'dress'): True, ('ok', 'partner'): True, ('outstanding', 'sorbet'): True, ('overall', 'great'): True, ('partner', 'salmon'): True, ('photo', 'overall'): True, ('point', 'also'): True, ('reservation', '5:30'): True, ('salmon', 'awesome

In [233]:
import random
random.shuffle(train_data2)
print(len(train_data2))

500


In [234]:
train_set, test_set = train_data2[350:], train_data2[:150]

In [235]:
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

nbrefset = collections.defaultdict(set)
nbtestset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    nbrefset[label].add(i)
    observed = nb_classifier.classify(feats)
    nbtestset[observed].add(i)
print("BigramNB Recall")
print('neg recall:', recall(nbtestset['neg'], nbrefset['neg']))
nb_classifier.show_most_informative_features(n=5)

BigramNB Recall
neg recall: 0.7733333333333333
Most Informative Features
     ('creme', 'brulee') = True              pos : neg    =      4.4 : 1.0
      ('pretty', 'good') = True              pos : neg    =      4.4 : 1.0
        ('feel', 'like') = True              pos : neg    =      3.4 : 1.0
       ('day', 'hiking') = True              pos : neg    =      3.4 : 1.0
        ('top', 'notch') = True              pos : neg    =      3.4 : 1.0


In [206]:
from nltk.classify import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier.train(train_set, 
                                             binary=True, 
                                             entropy_cutoff=0.8, 
                                             depth_cutoff=5, 
                                             support_cutoff=30)
refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = dt_classifier.classify(feats)
    testset[observed].add(i)
print("BigramDT Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

BigramDT Recall
neg recall: 0.6044776119402985


In [207]:
from nltk.classify import MaxentClassifier

logit_classifier = MaxentClassifier.train(train_set, algorithm='gis', trace=0, max_iter=10, min_lldelta=0.5)

refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = logit_classifier.classify(feats)
    testset[observed].add(i)
print("BigramsLogit Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

BigramsLogit Recall
neg recall: 0.7157894736842105


In [208]:
SVM_classifier = SklearnClassifier(SVC(), sparse=False).train(train_set)
refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = SVM_classifier.classify(feats)
    testset[observed].add(i)
    
print("BigramsSVM Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

BigramsSVM Recall
neg recall: 0.6066666666666667


In [209]:
from nltk import bigrams, trigrams
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

def bag_of_trigrams_words(words, score_fn=TrigramAssocMeasures.chi_sq, n=200):
    Trigram_finder = TrigramCollocationFinder.from_words(words)  
    Trigrams = Trigram_finder.nbest(score_fn, n)  
    return bag_of_words(Trigrams)

In [210]:
combined = zip(reviews,ratings1)

In [211]:
train_data3 =[]

for z, e in combined:
    bag_of_trigrams_words(z)
    train_data3.append((bag_of_trigrams_words(z),e))
    
random.shuffle(train_data3)
print(len(train_data3))

500


In [212]:
train_set, test_set = train_data3[350:], train_data3[:150]

In [213]:
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

nbrefset = collections.defaultdict(set)
nbtestset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    nbrefset[label].add(i)
    observed = nb_classifier.classify(feats)
    nbtestset[observed].add(i)
print("TrigramNB Recall")
print('neg recall:', recall(nbtestset['neg'], nbrefset['neg']))
print("")

from nltk.classify import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier.train(train_set, 
                                             binary=True, 
                                             entropy_cutoff=0.8, 
                                             depth_cutoff=5, 
                                             support_cutoff=30)
refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = dt_classifier.classify(feats)
    testset[observed].add(i)
print("TrigramDT Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))
print("")

from nltk.classify import MaxentClassifier

logit_classifier = MaxentClassifier.train(train_set, algorithm='gis', trace=0, max_iter=10, min_lldelta=0.5)

refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = logit_classifier.classify(feats)
    testset[observed].add(i)
print("TrigramsLogit Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))
print("")

TrigramNB Recall
neg recall: 0.5339805825242718

TrigramDT Recall
neg recall: 0.528169014084507

TrigramsLogit Recall
neg recall: 0.6206896551724138



In [214]:
SVM_classifier = SklearnClassifier(SVC(), sparse=False).train(train_set)
refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = SVM_classifier.classify(feats)
    testset[observed].add(i)
    
print("TrigramsSVM Recall")
print('neg recall:', recall(testset['neg'], refset['neg']))

TrigramsSVM Recall
neg recall: 0.5333333333333333


In [226]:
def bigrams_words(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    
    bigram_finder = BigramCollocationFinder.from_words(words)  
    
    bigrams = bigram_finder.nbest(score_fn, n)  
    
    return bigrams

def trigrams_words(words, score_fn=TrigramAssocMeasures.chi_sq, n=200):
    Trigram_finder = TrigramCollocationFinder.from_words(words)  
    Trigrams = Trigram_finder.nbest(score_fn, n)  
    return trigrams

def bag_of_Ngrams_words(words):
    
    bigramBag = bigrams_words(words)
    
    for b in range(len(reviews)):
        bigramBag[b]=' '.join(bigramBag[b])
        
    trigramBag = trigrams_words(words)
    
    for t in range(len(reviews)):
        trigramBag[t]=' '.join(trigramBag[t])
        
    return bag_of_words(trigramBag + bigramBag + words)

In [228]:
combined = zip(reviews,ratings1)

In [229]:
train_data4 =[]

for z, e in combined:
    bag_of_Ngrams_words(z)
    train_data4.append((bag_of_Ngrams_words(z),e))
    
random.shuffle(train_data4)
print(len(train_data4))

IndexError: list index out of range